In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [2]:
## Read teh pdfs from the folder
loader = PyPDFDirectoryLoader("./us_census/")

documents = loader.load()
text_splitter= RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'source': 'us_census/acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under \nthe Continuous Enrollment Provision.2 The American \nRescue Plan (ARP) enhanced Marketplace premium \nsubsidies for those with incomes above 400 percent \nof the poverty level as well as for unemployed people.3\nIn addition to national policies, individual states and \nthe District of Columbia can affect h

In [3]:
len(final_documents)

316

In [4]:
## Embedding Using Huggingface
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name = "BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'mps'},
    encode_kwargs = {'normalize_embeddings':'True'}
)

/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import numpy as np
np.array(huggingface_embeddings.embed_query(final_documents[0].page_content))

array([-0.07903478, -0.01134118, -0.02312095,  0.02844459,  0.05053345,
        0.05317833, -0.01907798,  0.03456025, -0.1021137 , -0.02915704,
        0.08524267,  0.05650731, -0.02545437, -0.03308494, -0.00635745,
        0.04090864, -0.00628109,  0.00356741, -0.03854131,  0.03667689,
       -0.04289804,  0.03425252, -0.031169  , -0.03793728,  0.01728389,
        0.01214924,  0.00653116,  0.01463565, -0.05529063, -0.15320699,
        0.00730851,  0.03202941, -0.04701129, -0.01595969,  0.0187444 ,
        0.02642934, -0.02306372,  0.08438037,  0.04182491,  0.05278177,
       -0.03057604,  0.01564259, -0.01689073,  0.00529407, -0.02417439,
        0.00412995, -0.01889943, -0.00150627, -0.00836937, -0.03390067,
        0.03515955, -0.00553136,  0.04910935,  0.05971861,  0.05615967,
       -0.05105153,  0.01475138, -0.01849965, -0.03284644,  0.03576624,
        0.04947709, -0.00938883, -0.26202121,  0.0975034 ,  0.01715687,
        0.04781387, -0.00556323, -0.0029831 , -0.02207349, -0.04

In [6]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [7]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [8]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x317ef97b0> search_kwargs={'k': 3}


In [9]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=""

In [10]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

/var/folders/_c/_wx3_r550ylgdgq0h4q8sllw0000gn/T/ipykernel_2599/386166796.py:3: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  hf=HuggingFaceHub(


'What is the health insurance coverage?\n\nThe health insurance coverage is a health insurance policy that is provided by the employer to the employee. The employer pays the premiums for the health insurance coverage. The employee is responsible for paying the deductible and co-payments.\n\nWhat is the health insurance coverage?\n\nThe health insurance coverage is a health insurance policy that is provided by the employer to the employee. The employer pays the premiums for the health insurance coverage. The employee is responsible for paying the deduct'

In [ ]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

In [12]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [13]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [14]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [15]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [16]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
